In [1]:
import random

In [31]:
class PairLink:
    
    def __init__(self, key, value, unwanted = []):
        self._unwanted = unwanted
        self._val = value
        self._key = key
    
    def clean(self, f):
        def alter(elt):
            if type(elt) != str: return None
            for item in self._unwanted:
                if type(elt) == str:
                    elt = elt.replace(item, '')
            return f(elt)
        self._val = alter(self._val)
        self._key = alter(self._key)
    
    def val(self):
        return self._val
    
    def key(self):
        return self._key
    
    def __repr__(self):
        return str(self._key) + ' -> ' + str(self._val)

In [3]:
class MkvPool:
    
    def __init__(self, pair_link_list):
        self._list = [l for l in pair_link_list]
    
    def clean_all(self, f):
        return [l.clean(f) for l in self._list]
    
    def item_list(self):
        return [item for item in dic(self).keys()]
    
    def size(self):
        return len(self._list)
    
    def lst(self):
        return self._list
    
    def dic(self):
        return {l.key(): l.val() for l in self._list}
    
    def next_deterministic(self, first):
        k = {}
        for l in self.lst():
            if l.key() == first:
                if l.val() not in k.keys():
                    k[l.val()] = 1
                else:
                    k[l.val()] += 1
        return max(k, key = k.get)

In [32]:
def read_txt(file, characters = None):
    finn = open(file)
    data = finn.read(characters)
    finn.close()
    return data

def word_PairLinks(string, unwanted_chars):
    words = string.split()
    links = [PairLink(None, words[0], unwanted_chars)]
    for i in range(len(words) - 2):
        links.append(PairLink(words[i], words[i + 1], unwanted_chars))
    links.append(PairLink(words[-1], None))
    return links

unwanted = ['"', '^', '*', '-', '_', '/', '[', ']', '<', '>', '~', ',']
txt = read_txt('data/huck_finn.txt', 500000)
word_pairs = word_PairLinks(txt, unwanted)
pool = MkvPool(word_pairs)
pool.clean_all(lambda s: s.lower())

pooldict = pool.dic()

In [35]:
def make_deterministic_chain(pool, length, seed = None):
    seen = []
    counter = 0
    last = seed
    chain = '' if not seed else seed
    while counter < length:
        nxt = pool.next_deterministic(last)
        seen += nxt
        chain += ' ' + nxt
        last = nxt
        counter += 1
    return chain

def make_stochastic_chain(dictionary, length, seed = None):
    seen = []
    counter = 0
    last = seed
    chain = '' if not seed else seed
    while counter < length:
        nxt = dictionary[last]
        chain += ' ' + nxt
        last = nxt
        counter += 1
    return chain

In [44]:
print(make_deterministic_chain(pool, 15, 'would'))

would be a little and the king he was a little and the king he was
